<h1>Explore and cluster the neighborhoods in Toronto (Part II).</h1>

<i>Part I code

In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text
soup = BeautifulSoup(source, 'lxml')
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned': #clear dataframe
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


<i>Creat a funtion to get GEO_CODE by POSTAL_CODE

In [13]:
#!pip install geocoder
import geocoder # import geocoder

In [42]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
          g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
          lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude


In [44]:
print(get_geocode('M1B'))

<i> Create a geocode table called df2

In [15]:
column_names=['Postalcode','Latitude','Longtitude']
df2 = pd.DataFrame(columns=column_names)
df2.head()

,Postalcode,Latitude,Longtitude


In [ ]:
n=5#len(df.index)
i=0
while i<n:
    pc=df.loc[i,'PostalCode']
    result=get_geocode(pc)
    pd.concat([pd.DataFrame([[pc,result[0],result[1]]], columns=column_names), df2])
    i+=1
df2.head()


<i>The google get_geocode doesn't work so i have to download the csv file, thanks to google :)

In [50]:
#geo_df=pd.read_csv('http://cocl.us/Geospatial_data') #in case you have a good internet =)))))
geo_df=pd.read_csv('C:\Khoa\DataScience_Learning\Geospatial_Coordinates.csv')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo_merged = pd.merge(geo_df, df, on='PostalCode')

In [54]:
geo_data=geo_merged[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
